# 总览




## 1.按月份所有城市平均出发/到达价格和起降数（city_avg_data)
## 2.全国按时间，无城市，平均到达价格-起降数
## 3.全国按城市，无时间，平均出发/到达价格
## 4.城市在1月到7月每10天的平均(到达)机票价格 (city_tenday_data)
## 5.每月全国航班数量/平均到达价格
## 6.一天内不同时间的机票平均价格
## 7.每天每个城市起降数列表 (city_flightnum_byday)
## 8.每个城市平均最低价格及所在最多的时间点

# 下载数据

In [5]:
import findspark
import time
from pyspark.sql import SQLContext, SparkSession

findspark.init()
spark = SparkSession.Builder().appName('sql').master('local').getOrCreate()

In [6]:
prop = {
    'user':'root',
    'password':'1',
    'driver':'com.mysql.jdbc.Driver'
}

url = 'jdbc:mysql://localhost:3306/flight'
df = spark.read.jdbc(url=url, table='flight_split', properties=prop)
print('connect success')

connect success


In [4]:
df.write.json("file:///g:/json")

# 处理数据

### 读数据

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType
import json

In [2]:
sparkSession = SparkSession.builder.getOrCreate()
df = sparkSession.read.json("./final_flight_split.json")

In [3]:
df.show(3)

+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+--------------------+
|arr_ap|arr_ct|   arr_tm| company|dep_ap|dep_ct|   dep_tm|   discount|flightNum| flight_dt|plane_sz| plane_type|price|    qry_dt|            rec_uuid|
+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+--------------------+
|      |  三亚| 00:00:00|吉祥航空|      |  上海| 00:00:00|3.1折经济舱|   HO1129|2019-09-12|      中|   空客A321|730.0|2019-09-11|2c0f8dab-56ff-4ea...|
|      |  三亚| 00:00:00|四川航空|      |  重庆| 00:00:00|3.1折经济舱|   3U8715|2019-09-12|      中|   空客A320|560.0|2019-09-11|7f3ff21f-5d3a-4a0...|
|      |  三亚| 00:00:00|山东航空|      |  济南| 00:00:00|2.3折经济舱|   SC4751|2019-09-13|      中|波音737-800|640.0|2019-09-12|cc08d14f-f89d-4eb...|
+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+----------

### 筛选有效信息

In [15]:
da = df.drop("dep_tm").drop("arr_tm").drop("rec_uuid").drop("arr_ap").drop("dep_ap").drop("discount").drop("plane_type").drop("qry_dt")
da = da.drop("plane_sz")
da = da.withColumn("line", f.concat_ws('-',da['company'],da['flightNum']))
da = da.select("arr_ct","dep_ct","company","flightNum","line","flight_dt","price")
da.show(5)

+------+------+--------+---------+---------------+----------+-----+
|arr_ct|dep_ct| company|flightNum|           line| flight_dt|price|
+------+------+--------+---------+---------------+----------+-----+
|  三亚|  上海|吉祥航空|   HO1129|吉祥航空-HO1129|2019-09-12|730.0|
|  三亚|  重庆|四川航空|   3U8715|四川航空-3U8715|2019-09-12|560.0|
|  三亚|  济南|山东航空|   SC4751|山东航空-SC4751|2019-09-13|640.0|
|  三亚|  温州|深圳航空|   ZH9723|深圳航空-ZH9723|2019-09-21|530.0|
|  三亚|  重庆|西部航空|   PN6213|西部航空-PN6213|2019-09-12|530.0|
+------+------+--------+---------+---------------+----------+-----+
only showing top 5 rows



## 1.按月份所有城市平均出发/到达价格和起降数

### 月份—城市—到达平均价格—出发平均价格—起飞数—降落数—总起降数

In [16]:
da_2 = da.select("*",f.split(da.flight_dt,"-").alias("sp_dt")).drop("flight_dt")
da_2.show(5)

+------+------+--------+---------+---------------+-----+--------------+
|arr_ct|dep_ct| company|flightNum|           line|price|         sp_dt|
+------+------+--------+---------+---------------+-----+--------------+
|  三亚|  上海|吉祥航空|   HO1129|吉祥航空-HO1129|730.0|[2019, 09, 12]|
|  三亚|  重庆|四川航空|   3U8715|四川航空-3U8715|560.0|[2019, 09, 12]|
|  三亚|  济南|山东航空|   SC4751|山东航空-SC4751|640.0|[2019, 09, 13]|
|  三亚|  温州|深圳航空|   ZH9723|深圳航空-ZH9723|530.0|[2019, 09, 21]|
|  三亚|  重庆|西部航空|   PN6213|西部航空-PN6213|530.0|[2019, 09, 12]|
+------+------+--------+---------+---------------+-----+--------------+
only showing top 5 rows



In [17]:
da_3 = da_2.withColumn("cnt",f.abs(da_2.price-da_2.price+1))
da_3.show(5)

+------+------+--------+---------+---------------+-----+--------------+---+
|arr_ct|dep_ct| company|flightNum|           line|price|         sp_dt|cnt|
+------+------+--------+---------+---------------+-----+--------------+---+
|  三亚|  上海|吉祥航空|   HO1129|吉祥航空-HO1129|730.0|[2019, 09, 12]|1.0|
|  三亚|  重庆|四川航空|   3U8715|四川航空-3U8715|560.0|[2019, 09, 12]|1.0|
|  三亚|  济南|山东航空|   SC4751|山东航空-SC4751|640.0|[2019, 09, 13]|1.0|
|  三亚|  温州|深圳航空|   ZH9723|深圳航空-ZH9723|530.0|[2019, 09, 21]|1.0|
|  三亚|  重庆|西部航空|   PN6213|西部航空-PN6213|530.0|[2019, 09, 12]|1.0|
+------+------+--------+---------+---------------+-----+--------------+---+
only showing top 5 rows



### 出发数据

In [18]:
ds = da_3.withColumn('month',da_3["sp_dt"].getItem(1)).drop("sp_dt").drop("line").drop("arr_ct").drop("company").drop("flightNum")
ds = ds.select("month","dep_ct","price","cnt")
ds.show(5)

+-----+------+-----+---+
|month|dep_ct|price|cnt|
+-----+------+-----+---+
|   09|  上海|730.0|1.0|
|   09|  重庆|560.0|1.0|
|   09|  济南|640.0|1.0|
|   09|  温州|530.0|1.0|
|   09|  重庆|530.0|1.0|
+-----+------+-----+---+
only showing top 5 rows



In [19]:
ds_2 = ds.groupBy("month","dep_ct").agg({"price":"sum","cnt":"sum"})
ds_2 = ds_2.select("*",ds_2['sum(price)']/ds_2['sum(cnt)'])
ds_2 = ds_2.select("*",f.bround("(sum(price) / sum(cnt))", scale=2).alias('avg_dep_price')).drop("(sum(price) / sum(cnt))").drop("sum(price)")
ds_2 = ds_2.withColumnRenamed("sum(cnt)","sum_dep")
ds_2.show(5)

+-----+--------+-------+-------------+
|month|  dep_ct|sum_dep|avg_dep_price|
+-----+--------+-------+-------------+
|   12|    大连| 1188.0|       631.46|
|   08|乌鲁木齐| 1648.0|       2297.2|
|   05|    南昌| 2218.0|       502.17|
|   06|    昆明| 8639.0|       473.18|
|   07|    昆明| 4588.0|       1079.4|
+-----+--------+-------+-------------+
only showing top 5 rows



#### 7月cnt乘2 8月~12月cnt乘4

In [20]:
ds_3 = ds_2.withColumn('new_dep',f.when(ds_2['month'] == '07',ds_2['sum_dep']*2.38).when((ds_2['month'] <= '13')& \
                        ('07' < ds_2['month']),ds_2['sum_dep']*4.28).otherwise(ds_2['sum_dep']))
ds_3.show(5)

+-----+--------+-------+-------------+------------------+
|month|  dep_ct|sum_dep|avg_dep_price|           new_dep|
+-----+--------+-------+-------------+------------------+
|   12|    大连| 1188.0|       631.46|           5084.64|
|   08|乌鲁木齐| 1648.0|       2297.2|7053.4400000000005|
|   05|    南昌| 2218.0|       502.17|            2218.0|
|   06|    昆明| 8639.0|       473.18|            8639.0|
|   07|    昆明| 4588.0|       1079.4|10919.439999999999|
+-----+--------+-------+-------------+------------------+
only showing top 5 rows



In [21]:
# 标准化
ds_4 = ds_3.drop("sum_dep")
ds_4 = ds_4.withColumn("sum_dep", ds_4["new_dep"].cast(IntegerType())).drop("new_dep")
ds_4 = ds_4.withColumnRenamed("dep_ct","city") # 方便合并
ds_4.show(5)

+-----+--------+-------------+-------+
|month|    city|avg_dep_price|sum_dep|
+-----+--------+-------------+-------+
|   12|    大连|       631.46|   5084|
|   08|乌鲁木齐|       2297.2|   7053|
|   05|    南昌|       502.17|   2218|
|   06|    昆明|       473.18|   8639|
|   07|    昆明|       1079.4|  10919|
+-----+--------+-------------+-------+
only showing top 5 rows



### 到达数据

In [22]:
dt = da_3.withColumn('month',da_3["sp_dt"].getItem(1)).drop("sp_dt").drop("line").drop("dep_ct").drop("company").drop("flightNum")
dt = dt.select("month","arr_ct","price","cnt")
dt.show(5)

+-----+------+-----+---+
|month|arr_ct|price|cnt|
+-----+------+-----+---+
|   09|  三亚|730.0|1.0|
|   09|  三亚|560.0|1.0|
|   09|  三亚|640.0|1.0|
|   09|  三亚|530.0|1.0|
|   09|  三亚|530.0|1.0|
+-----+------+-----+---+
only showing top 5 rows



In [23]:
dt_2 = dt.groupBy("month","arr_ct").agg({"price":"sum","cnt":"sum"})
dt_2 = dt_2.select("*",dt_2['sum(price)']/dt_2['sum(cnt)'])
dt_2 = dt_2.select("*",f.bround("(sum(price) / sum(cnt))", scale=2).alias('avg_arr_price')).drop("(sum(price) / sum(cnt))").drop("sum(price)")
dt_2 = dt_2.withColumnRenamed("sum(cnt)","sum_arr")
dt_2.show(5)

+-----+--------+-------+-------------+
|month|  arr_ct|sum_arr|avg_arr_price|
+-----+--------+-------+-------------+
|   08|乌鲁木齐| 1504.0|      1894.25|
|   12|    大连| 1255.0|       617.03|
|   06|    昆明| 8416.0|       500.37|
|   07|    昆明| 4587.0|      1120.68|
|   05|    南昌| 2377.0|       498.64|
+-----+--------+-------+-------------+
only showing top 5 rows



#### 7月cnt乘2 8月~12月cnt乘4

In [24]:
dt_3 = dt_2.withColumn('new_arr',f.when(dt_2['month'] == '07',dt_2['sum_arr']*2.38).when((dt_2['month'] <= '13')& \
                        ('07' < dt_2['month']),dt_2['sum_arr']*4.28).otherwise(dt_2['sum_arr']))
dt_3.show(5)

+-----+--------+-------+-------------+-----------------+
|month|  arr_ct|sum_arr|avg_arr_price|          new_arr|
+-----+--------+-------+-------------+-----------------+
|   08|乌鲁木齐| 1504.0|      1894.25|6437.120000000001|
|   12|    大连| 1255.0|       617.03|5371.400000000001|
|   06|    昆明| 8416.0|       500.37|           8416.0|
|   07|    昆明| 4587.0|      1120.68|         10917.06|
|   05|    南昌| 2377.0|       498.64|           2377.0|
+-----+--------+-------+-------------+-----------------+
only showing top 5 rows



In [25]:
# 标准化
dt_4 = dt_3.drop("sum_arr")
dt_4 = dt_4.withColumn("sum_arr", dt_4["new_arr"].cast(IntegerType())).drop("new_arr")
dt_4 = dt_4.withColumnRenamed("arr_ct","city") # 方便合并
dt_4.show(5)

+-----+--------+-------------+-------+
|month|    city|avg_arr_price|sum_arr|
+-----+--------+-------------+-------+
|   08|乌鲁木齐|      1894.25|   6437|
|   12|    大连|       617.03|   5371|
|   06|    昆明|       500.37|   8416|
|   07|    昆明|      1120.68|  10917|
|   05|    南昌|       498.64|   2377|
+-----+--------+-------------+-------+
only showing top 5 rows



### 两表合并

In [26]:
final = dt_4.join(ds_4,["month","city"],"full")
final = final.orderBy("month")
final.show(5)

+-----+----+-------------+-------+-------------+-------+
|month|city|avg_arr_price|sum_arr|avg_dep_price|sum_dep|
+-----+----+-------------+-------+-------------+-------+
|   01|三亚|      2464.64|   5486|      1422.06|   6787|
|   01|合肥|       905.17|   3655|       915.09|   3126|
|   01|大连|      1148.26|   5164|       892.18|   4816|
|   01|济南|      1017.53|   4672|       1071.3|   4159|
|   01|青岛|       837.07|   5690|       903.83|   5014|
+-----+----+-------------+-------+-------------+-------+
only showing top 5 rows



In [28]:
final = final.withColumn("sum_dep_arr",f.abs(final.sum_dep+final.sum_arr))
final = final.select("month","city","avg_dep_price","avg_arr_price","sum_dep","sum_arr","sum_dep_arr")
final.show(5)

+-----+----+-------------+-------------+-------+-------+-----------+
|month|city|avg_dep_price|avg_arr_price|sum_dep|sum_arr|sum_dep_arr|
+-----+----+-------------+-------------+-------+-------+-----------+
|   01|三亚|      1422.06|      2464.64|   6787|   5486|      12273|
|   01|济南|       1071.3|      1017.53|   4159|   4672|       8831|
|   01|大连|       892.18|      1148.26|   4816|   5164|       9980|
|   01|合肥|       915.09|       905.17|   3126|   3655|       6781|
|   01|青岛|       903.83|       837.07|   5014|   5690|      10704|
+-----+----+-------------+-------------+-------+-------+-----------+
only showing top 5 rows



### 写数据库（city_avg_data)

In [ ]:
prop = {
    'user':'root',
    'password':'1',
    'driver':'com.mysql.jdbc.Driver'
}
url = 'jdbc:mysql://localhost:3306/flight?useUnicode=true&characterEncoding=utf8'

In [ ]:
final.write.jdbc(url, 'city_avg_data', properties=prop)

## 2.全国按时间，无城市，平均到达价格-起降数

In [ ]:
sd = final.withColumn("cnt",f.abs(final.sum_dep-final.sum_dep+1))
sd = sd.drop("sum_price").drop("sum_arr").drop("sum_dep")
sd.show(5)

In [ ]:
sd_2 = sd.groupBy("month").agg({"sum_dep_arr":"sum","cnt":"sum","avg_arr_price":"sum"})
sd_2.show(5)

In [ ]:
sd_3 = sd_2.select("*",sd_2['sum(avg_arr_price)']/sd_2['sum(cnt)'])
sd_3 = sd_3.drop('sum(avg_arr_price)').drop('sum(cnt)')
sd_3 = sd_3.withColumnRenamed("(sum(avg_arr_price) / sum(cnt))","avg_price")
sd_3 = sd_3.withColumnRenamed("sum(sum_dep_arr)","sum_dep_arr")
sd_3 = sd_3.select("*",f.bround("avg_price", scale=2).alias("avg_arr_price")).drop("avg_price")
sd_3.show(5)

In [ ]:
sd_3_c = sd_3.collect()

In [ ]:
sd_ans = []
for i in range(len(sd_3_c)):
    d = {}
    d['month'] = sd_3_c[i][0]
    d['sum_dep_arr'] = sd_3_c[i][1]
    d['avg_arr_price'] = sd_3_c[i][2]
    sd_ans.append(d)
sd_ans

## 3.全国按城市，无时间，平均出发/到达价格

In [29]:
dx = final.withColumn("cnt",f.abs(final.sum_dep-final.sum_dep+1))
dx = dx.groupBy("city").agg({"avg_dep_price":"sum","cnt":"sum","avg_arr_price":"sum"})
dx.show(5)

+----+------------------+--------+------------------+
|city|sum(avg_dep_price)|sum(cnt)|sum(avg_arr_price)|
+----+------------------+--------+------------------+
|南京| 8464.109999999999|      12|           9038.33|
|长沙|           9052.29|      12|           8315.91|
|沈阳|          11349.69|      12|          10938.55|
|西安| 9132.340000000002|      12| 8548.230000000001|
|无锡|          10449.18|      12|          10630.06|
+----+------------------+--------+------------------+
only showing top 5 rows



In [30]:
dx_2 = dx.withColumn("avg_dep_price",dx["sum(avg_dep_price)"]/dx["sum(cnt)"])
dx_2 = dx_2.withColumn("avg_arr_price",dx_2["sum(avg_arr_price)"]/dx_2["sum(cnt)"])
dx_2 = dx_2.select("city",f.bround("avg_arr_price", scale=2).alias("arr_price"),f.bround("avg_dep_price", scale=2).alias("dep_price"))
dx_2.show(5)

+----+---------+---------+
|city|arr_price|dep_price|
+----+---------+---------+
|南京|   753.19|   705.34|
|长沙|   692.99|   754.36|
|沈阳|   911.55|   945.81|
|西安|   712.35|   761.03|
|无锡|   885.84|   870.76|
+----+---------+---------+
only showing top 5 rows



### 出发

In [31]:
dx_d = dx_2.orderBy(-dx_2["dep_price"])
dx_d_c = dx_d.collect()
dx_d.show(5)

+--------+---------+---------+
|    city|arr_price|dep_price|
+--------+---------+---------+
|乌鲁木齐|  1367.75|  1395.44|
|    拉萨|  1262.04|  1378.31|
|    三亚|  1109.39|  1074.34|
|    北京|  1092.69|  1071.02|
|  哈尔滨|   991.06|   989.54|
+--------+---------+---------+
only showing top 5 rows



In [32]:
dx_ans_dep = []
for i in range(42):
    d = {}
    d['city'] = dx_d_c[i][0]
    d['price'] = dx_d_c[i][2]
    dx_ans_dep.append(d)
dx_ans_dep

[{'city': '乌鲁木齐', 'price': 1395.44},
 {'city': '拉萨', 'price': 1378.31},
 {'city': '三亚', 'price': 1074.34},
 {'city': '北京', 'price': 1071.02},
 {'city': '哈尔滨', 'price': 989.54},
 {'city': '长春', 'price': 972.69},
 {'city': '沈阳', 'price': 945.81},
 {'city': '深圳', 'price': 929.26},
 {'city': '广州', 'price': 923.63},
 {'city': '西宁', 'price': 912.55},
 {'city': '成都', 'price': 909.84},
 {'city': '无锡', 'price': 870.76},
 {'city': '上海', 'price': 858.66},
 {'city': '兰州', 'price': 857.99},
 {'city': '呼和浩特', 'price': 851.74},
 {'city': '济南', 'price': 837.05},
 {'city': '扬州', 'price': 835.91},
 {'city': '昆明', 'price': 834.59},
 {'city': '福州', 'price': 829.44},
 {'city': '武汉', 'price': 821.6},
 {'city': '海口', 'price': 819.22},
 {'city': '天津', 'price': 801.85},
 {'city': '珠海', 'price': 795.58},
 {'city': '银川', 'price': 790.52},
 {'city': '贵阳', 'price': 786.3},
 {'city': '太原', 'price': 785.32},
 {'city': '大连', 'price': 783.58},
 {'city': '郑州', 'price': 764.51},
 {'city': '西安', 'price': 761.03},
 {'city

### 到达

In [33]:
dx_a = dx_2.orderBy(-dx_2["arr_price"])
dx_a_c = dx_a.collect()
dx_a.show(5)

+--------+---------+---------+
|    city|arr_price|dep_price|
+--------+---------+---------+
|乌鲁木齐|  1367.75|  1395.44|
|    拉萨|  1262.04|  1378.31|
|    三亚|  1109.39|  1074.34|
|    北京|  1092.69|  1071.02|
|    深圳|   993.53|   929.26|
+--------+---------+---------+
only showing top 5 rows



In [34]:
dx_ans_arr = []
for i in range(42):
    d = {}
    d['city'] = dx_a_c[i][0]
    d['price'] = dx_a_c[i][1]
    dx_ans_arr.append(d)
dx_ans_arr

[{'city': '乌鲁木齐', 'price': 1367.75},
 {'city': '拉萨', 'price': 1262.04},
 {'city': '三亚', 'price': 1109.39},
 {'city': '北京', 'price': 1092.69},
 {'city': '深圳', 'price': 993.53},
 {'city': '哈尔滨', 'price': 991.06},
 {'city': '广州', 'price': 960.16},
 {'city': '西宁', 'price': 958.36},
 {'city': '长春', 'price': 953.88},
 {'city': '沈阳', 'price': 911.55},
 {'city': '扬州', 'price': 906.31},
 {'city': '上海', 'price': 888.33},
 {'city': '无锡', 'price': 885.84},
 {'city': '海口', 'price': 871.7},
 {'city': '成都', 'price': 861.31},
 {'city': '呼和浩特', 'price': 858.5},
 {'city': '昆明', 'price': 848.92},
 {'city': '福州', 'price': 842.4},
 {'city': '珠海', 'price': 828.32},
 {'city': '天津', 'price': 812.9},
 {'city': '武汉', 'price': 795.23},
 {'city': '厦门', 'price': 791.76},
 {'city': '杭州', 'price': 789.72},
 {'city': '济南', 'price': 787.51},
 {'city': '兰州', 'price': 760.88},
 {'city': '大连', 'price': 760.6},
 {'city': '泉州', 'price': 755.01},
 {'city': '南京', 'price': 753.19},
 {'city': '南宁', 'price': 744.24},
 {'city': 

## 4.城市在1月到7月每10天的平均(到达)机票价格 

### 城市—价格列表<时间，价格>(每月的5号，15号，25号)

In [5]:
da.show(3)

+------+------+--------+---------+---------------+----------+-----+
|arr_ct|dep_ct| company|flightNum|           line| flight_dt|price|
+------+------+--------+---------+---------------+----------+-----+
|  三亚|  上海|吉祥航空|   HO1129|吉祥航空-HO1129|2019-09-12|730.0|
|  三亚|  重庆|四川航空|   3U8715|四川航空-3U8715|2019-09-12|560.0|
|  三亚|  济南|山东航空|   SC4751|山东航空-SC4751|2019-09-13|640.0|
+------+------+--------+---------+---------------+----------+-----+
only showing top 3 rows



In [37]:
da_2 = da.select("arr_ct","flight_dt","price",f.split(df.flight_dt,"-").alias("sp_tm"))
da_2.show(5)

+------+----------+-----+--------------+
|arr_ct| flight_dt|price|         sp_tm|
+------+----------+-----+--------------+
|  三亚|2019-09-12|730.0|[2019, 09, 12]|
|  三亚|2019-09-12|560.0|[2019, 09, 12]|
|  三亚|2019-09-13|640.0|[2019, 09, 13]|
|  三亚|2019-09-21|530.0|[2019, 09, 21]|
|  三亚|2019-09-12|530.0|[2019, 09, 12]|
+------+----------+-----+--------------+
only showing top 5 rows



In [38]:
da_3 = da_2.select("arr_ct","price","flight_dt",f.split(da_2.flight_dt,"-").alias("sp_dt"))
da_3 = da_3.withColumn("cnnt",f.abs(da_3.price-da_3.price+1))
da_3.show(5)

+------+-----+----------+--------------+----+
|arr_ct|price| flight_dt|         sp_dt|cnnt|
+------+-----+----------+--------------+----+
|  三亚|730.0|2019-09-12|[2019, 09, 12]| 1.0|
|  三亚|560.0|2019-09-12|[2019, 09, 12]| 1.0|
|  三亚|640.0|2019-09-13|[2019, 09, 13]| 1.0|
|  三亚|530.0|2019-09-21|[2019, 09, 21]| 1.0|
|  三亚|530.0|2019-09-12|[2019, 09, 12]| 1.0|
+------+-----+----------+--------------+----+
only showing top 5 rows



In [39]:
da_4 = da_3.withColumn('year',da_3["sp_dt"].getItem(0)).drop("line")
da_4 = da_4.withColumn('month',da_4["sp_dt"].getItem(1))
da_4 = da_4.withColumn('day',da_4["sp_dt"].getItem(2)).drop("sp_dt")
da_4.show(5)

+------+-----+----------+----+----+-----+---+
|arr_ct|price| flight_dt|cnnt|year|month|day|
+------+-----+----------+----+----+-----+---+
|  三亚|730.0|2019-09-12| 1.0|2019|   09| 12|
|  三亚|560.0|2019-09-12| 1.0|2019|   09| 12|
|  三亚|640.0|2019-09-13| 1.0|2019|   09| 13|
|  三亚|530.0|2019-09-21| 1.0|2019|   09| 21|
|  三亚|530.0|2019-09-12| 1.0|2019|   09| 12|
+------+-----+----------+----+----+-----+---+
only showing top 5 rows



#### 分类处理

In [40]:
dq = da_4.filter(da_4['year'] == '2020')
dq = dq.withColumn("flag", dq["day"].cast(IntegerType()))
dq = dq.withColumn("flag", dq["flag"]/10)
dq = dq.withColumn("flag", dq["flag"].cast(IntegerType()))
dq.show(5)

+------+------+----------+----+----+-----+---+----+
|arr_ct| price| flight_dt|cnnt|year|month|day|flag|
+------+------+----------+----+----+-----+---+----+
|  三亚| 250.0|2020-05-25| 1.0|2020|   05| 25|   2|
|  厦门| 360.0|2020-06-20| 1.0|2020|   06| 20|   2|
|  大连|2345.0|2020-01-14| 1.0|2020|   01| 14|   1|
|  太原|1050.0|2020-01-30| 1.0|2020|   01| 30|   3|
|  成都| 920.0|2020-01-13| 1.0|2020|   01| 13|   1|
+------+------+----------+----+----+-----+---+----+
only showing top 5 rows



In [10]:
dq_2 = dq.withColumn('new_date',f.when(dq['flag'] == 0,"05").when(dq['flag'] == 1,"15").\
                        when(dq['flag'] == 2,"25").otherwise("25"))
dq_2 = dq_2.drop("flag").drop("day")
dq_2.show(5)

+------+------+----------+----+----+-----+--------+
|arr_ct| price| flight_dt|cnnt|year|month|new_date|
+------+------+----------+----+----+-----+--------+
|  三亚| 250.0|2020-05-25| 1.0|2020|   05|      25|
|  厦门| 360.0|2020-06-20| 1.0|2020|   06|      25|
|  大连|2345.0|2020-01-14| 1.0|2020|   01|      15|
|  太原|1050.0|2020-01-30| 1.0|2020|   01|      25|
|  成都| 920.0|2020-01-13| 1.0|2020|   01|      15|
+------+------+----------+----+----+-----+--------+
only showing top 5 rows



In [11]:
dq_3 = dq_2.withColumn("time", f.concat_ws('-',dq_2['year'],dq_2['month'],dq_2['new_date']))
dq_3 = dq_3.drop("month").drop("year")
dq_3.show(5)

+------+------+----------+----+--------+----------+
|arr_ct| price| flight_dt|cnnt|new_date|      time|
+------+------+----------+----+--------+----------+
|  三亚| 250.0|2020-05-25| 1.0|      25|2020-05-25|
|  厦门| 360.0|2020-06-20| 1.0|      25|2020-06-25|
|  大连|2345.0|2020-01-14| 1.0|      15|2020-01-15|
|  太原|1050.0|2020-01-30| 1.0|      25|2020-01-25|
|  成都| 920.0|2020-01-13| 1.0|      15|2020-01-15|
+------+------+----------+----+--------+----------+
only showing top 5 rows



#### 选择1月25日~6月15日

In [12]:
dq_4 = dq_3.filter( ("2020-01-24"<dq_3["time"]) & (dq_3["time"]<"2020-06-16") )
dq_4.show(5)

+------+------+----------+----+--------+----------+
|arr_ct| price| flight_dt|cnnt|new_date|      time|
+------+------+----------+----+--------+----------+
|  三亚| 250.0|2020-05-25| 1.0|      25|2020-05-25|
|  太原|1050.0|2020-01-30| 1.0|      25|2020-01-25|
|  昆明| 410.0|2020-06-15| 1.0|      15|2020-06-15|
|  昆明| 260.0|2020-06-15| 1.0|      15|2020-06-15|
|  杭州| 660.0|2020-03-19| 1.0|      15|2020-03-15|
+------+------+----------+----+--------+----------+
only showing top 5 rows



In [13]:
dq_5 = dq_4.drop("flag")
dq_5 = dq_5.groupBy("time","arr_ct").agg({"cnnt":"sum","price":"sum"})
dq_5.show(5)

+----------+------+----------+---------+
|      time|arr_ct|sum(price)|sum(cnnt)|
+----------+------+----------+---------+
|2020-05-25|  重庆| 1156915.0|   2584.0|
|2020-06-05|  合肥|  375059.0|    785.0|
|2020-01-25|  北京| 8638024.0|   6862.0|
|2020-05-25|  南昌|  359612.0|    754.0|
|2020-03-25|  厦门| 1136901.0|   1836.0|
+----------+------+----------+---------+
only showing top 5 rows



In [14]:
dq_6 = dq_5.select("*",dq_5['sum(price)']/dq_5['sum(cnnt)'])
dq_6 = dq_6.drop('sum(price)').drop('sum(cnnt)')
dq_6 = dq_6.select("*",f.bround("(sum(price) / sum(cnnt))", scale=2).alias('avg_price')).drop("(sum(price) / sum(cnnt))")
dq_6.show(5)

+----------+------+---------+
|      time|arr_ct|avg_price|
+----------+------+---------+
|2020-05-25|  重庆|   447.72|
|2020-06-05|  合肥|   477.78|
|2020-01-25|  北京|  1258.82|
|2020-05-25|  南昌|   476.94|
|2020-03-25|  厦门|   619.23|
+----------+------+---------+
only showing top 5 rows



In [16]:
dq_6 = dq_6.orderBy("arr_ct")
dqc = dq_6.collect()
dqc[0]

Row(time='2020-05-15', arr_ct='三亚', avg_price=556.66)

In [18]:
qDict = {}
for i in range(len(dqc)) :
    if dqc[i][1] not in qDict:
        qDict[dqc[i][1]] = {dqc[i][0]:dqc[i][2]}
    else :
        qDict[dqc[i][1]][dqc[i][0]] = dqc[i][2]
    
for a in qDict :
    qDict[a] = sorted(qDict[a].items(),key=lambda item:item[0], reverse=False)

In [19]:
qDict['三亚']

[('2020-01-25', 2887.94),
 ('2020-02-05', 1133.59),
 ('2020-02-15', 1019.29),
 ('2020-02-25', 760.64),
 ('2020-03-05', 580.97),
 ('2020-03-15', 537.0),
 ('2020-03-25', 489.96),
 ('2020-04-05', 558.73),
 ('2020-04-15', 567.85),
 ('2020-04-25', 553.28),
 ('2020-05-05', 565.74),
 ('2020-05-15', 556.66),
 ('2020-05-25', 517.29),
 ('2020-06-05', 490.22),
 ('2020-06-15', 562.07)]

In [21]:
def qtoDB(Dict):
    final_Dict = []
    for key in Dict :
        td = {}
        td['city'] = key
        tl = []
        for s in range(len(Dict[key])):
            st = Dict[key][s][0] + '|' + f"{Dict[key][s][1]}"
            tl.append(st)
        td["min_price_list"] = f'{tl}'
        final_Dict.append(td)
    return final_Dict
qAns = qtoDB(qDict)

In [23]:
qAns[0]

{'city': '三亚',
 'min_price_list': "['2020-01-25|2887.94', '2020-02-05|1133.59', '2020-02-15|1019.29', '2020-02-25|760.64', '2020-03-05|580.97', '2020-03-15|537.0', '2020-03-25|489.96', '2020-04-05|558.73', '2020-04-15|567.85', '2020-04-25|553.28', '2020-05-05|565.74', '2020-05-15|556.66', '2020-05-25|517.29', '2020-06-05|490.22', '2020-06-15|562.07']"}

### 写数据库(city_tenday_data)

In [27]:
json_str = json.dumps(qAns,ensure_ascii=False)
with open('tmp_tenday_toDB.json', 'w',encoding='utf-8') as json_file:
    json_file.write(json_str)

In [28]:
dq_r = spark.read.json('tmp_tenday_toDB.json')
dq_r.show(5)

+--------+--------------------+
|    city|      min_price_list|
+--------+--------------------+
|    三亚|['2020-01-25|2887...|
|    上海|['2020-01-25|950....|
|乌鲁木齐|['2020-01-25|1997...|
|    兰州|['2020-01-25|1307...|
|    北京|['2020-01-25|1258...|
+--------+--------------------+
only showing top 5 rows



In [29]:
prop = {
    'user':'root',
    'password':'1',
    'driver':'com.mysql.jdbc.Driver'
}
url = 'jdbc:mysql://localhost:3306/flight?useUnicode=true&characterEncoding=utf8'

In [30]:
dq_r.write.jdbc(url, 'city_tenday_data', properties=prop)

## 5.每月全国航班数量/平均到达价格

In [18]:
dw = final.withColumn("cnt",f.abs(final.sum_dep-final.sum_dep+1))
dw = dw.drop("sum_price").drop("sum_arr").drop("sum_dep").drop("avg_dep_price")
dw.show(5)

+-----+----+-------------+-----------+---+
|month|city|avg_arr_price|sum_dep_arr|cnt|
+-----+----+-------------+-----------+---+
|   01|三亚|      2464.64|      12273|  1|
|   01|济南|      1017.53|       8831|  1|
|   01|大连|      1148.26|       9980|  1|
|   01|合肥|       905.17|       6781|  1|
|   01|青岛|       837.07|      10704|  1|
+-----+----+-------------+-----------+---+
only showing top 5 rows



In [19]:
dw_2 = dw.groupBy("month").agg({"sum_dep_arr":"sum","cnt":"sum","avg_arr_price":"sum"})
dw_2.show(5)

+-----+----------------+--------+------------------+
|month|sum(sum_dep_arr)|sum(cnt)|sum(avg_arr_price)|
+-----+----------------+--------+------------------+
|   01|          516996|      42|          48229.67|
|   02|          212844|      42| 43266.84000000001|
|   03|          344908|      42|          27380.04|
|   04|          298806|      42|          26767.96|
|   05|          331352|      42|          23013.86|
+-----+----------------+--------+------------------+
only showing top 5 rows



In [20]:
dw_3 = dw_2.select("*",dw_2['sum(avg_arr_price)']/dw_2['sum(cnt)'])
dw_3 = dw_3.drop('sum(avg_arr_price)').drop('sum(cnt)')
dw_3 = dw_3.withColumnRenamed("(sum(avg_arr_price) / sum(cnt))","avg_price")
dw_3 = dw_3.withColumnRenamed("sum(sum_dep_arr)","sum_dep_arr")
dw_3 = dw_3.select("*",f.bround("avg_price", scale=2).alias("avg_arr_price")).drop("avg_price")
dw_3.show(5)

+-----+-----------+-------------+
|month|sum_dep_arr|avg_arr_price|
+-----+-----------+-------------+
|   01|     516996|      1148.33|
|   02|     212844|      1030.16|
|   03|     344908|       651.91|
|   04|     298806|       637.33|
|   05|     331352|       547.95|
+-----+-----------+-------------+
only showing top 5 rows



In [21]:
dw_c = dw_3.collect()

In [23]:
wDict = []
for i in range(len(dw_c)):
    d = {}
    d['month'] = dw_c[i][0]
    d['sum_dep_arr'] = dw_c[i][1]
    d['avg_arr_price'] = dw_c[i][2]
    wDict.append(d)
wDict

[{'month': '01', 'sum_dep_arr': 516996, 'avg_arr_price': 1148.33},
 {'month': '02', 'sum_dep_arr': 212844, 'avg_arr_price': 1030.16},
 {'month': '03', 'sum_dep_arr': 344908, 'avg_arr_price': 651.91},
 {'month': '04', 'sum_dep_arr': 298806, 'avg_arr_price': 637.33},
 {'month': '05', 'sum_dep_arr': 331352, 'avg_arr_price': 547.95},
 {'month': '06', 'sum_dep_arr': 446988, 'avg_arr_price': 512.52},
 {'month': '07', 'sum_dep_arr': 549283, 'avg_arr_price': 1020.82},
 {'month': '08', 'sum_dep_arr': 521224, 'avg_arr_price': 1200.85},
 {'month': '09', 'sum_dep_arr': 682952, 'avg_arr_price': 845.36},
 {'month': '10', 'sum_dep_arr': 533777, 'avg_arr_price': 909.0},
 {'month': '11', 'sum_dep_arr': 508787, 'avg_arr_price': 805.0},
 {'month': '12', 'sum_dep_arr': 523050, 'avg_arr_price': 763.31}]

## 6.一天内不同时间的机票平均价格

In [24]:
df.show(3)

+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+--------------------+
|arr_ap|arr_ct|   arr_tm| company|dep_ap|dep_ct|   dep_tm|   discount|flightNum| flight_dt|plane_sz| plane_type|price|    qry_dt|            rec_uuid|
+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+--------------------+
|      |  三亚| 00:00:00|吉祥航空|      |  上海| 00:00:00|3.1折经济舱|   HO1129|2019-09-12|      中|   空客A321|730.0|2019-09-11|2c0f8dab-56ff-4ea...|
|      |  三亚| 00:00:00|四川航空|      |  重庆| 00:00:00|3.1折经济舱|   3U8715|2019-09-12|      中|   空客A320|560.0|2019-09-11|7f3ff21f-5d3a-4a0...|
|      |  三亚| 00:00:00|山东航空|      |  济南| 00:00:00|2.3折经济舱|   SC4751|2019-09-13|      中|波音737-800|640.0|2019-09-12|cc08d14f-f89d-4eb...|
+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+----------

In [30]:
dr = df.select("dep_tm","price",f.split(df.dep_tm,":").alias("sp_tm"))
dr = dr.withColumn('time',dr["sp_tm"].getItem(0)).drop("sp_tm")
dr.show(5)

+---------+-----+----+
|   dep_tm|price|time|
+---------+-----+----+
| 00:00:00|730.0|  00|
| 00:00:00|560.0|  00|
| 00:00:00|640.0|  00|
| 00:00:00|530.0|  00|
| 00:00:00|530.0|  00|
+---------+-----+----+
only showing top 5 rows



In [31]:
dr_2 = dr.withColumn("cnt",f.abs(dr.price-dr.price+1))
dr_2 = dr_2.groupBy("time").agg({"price":"sum","cnt":"sum"})
dr_2.show(5)

+----+-----------+--------+
|time| sum(price)|sum(cnt)|
+----+-----------+--------+
|  23|1.2357108E7| 22497.0|
|  12| 8.417455E7| 97857.0|
|  11|8.3693771E7| 94780.0|
|  16|8.3434657E7| 98234.0|
|  08| 8.685301E7|101798.0|
+----+-----------+--------+
only showing top 5 rows



In [32]:
dr_3 = dr_2.select("*",dr_2['sum(price)']/dr_2['sum(cnt)'])
dr_3 = dr_3.drop('sum(price)').drop('sum(cnt)')
dr_3 = dr_3.select("*",f.bround("(sum(price) / sum(cnt))", scale=2).alias('avg_price')).drop("(sum(price) / sum(cnt))")
dr_3.show(5)

+----+---------+
|time|avg_price|
+----+---------+
|  23|   549.28|
|  12|   860.18|
|  11|   883.03|
|  16|   849.35|
|  08|   853.19|
+----+---------+
only showing top 5 rows



In [33]:
dr_4 = dr_3.orderBy("time")
drc = dr_4.collect()

In [35]:
rDict = []
for i in range(len(drc)):
    d = {}
    d['tm'] = drc[i][0]
    d['p'] = drc[i][1]
    rDict.append(d)
rDict

[{'tm': ' 00', 'p': 810.85},
 {'tm': ' 01', 'p': 903.42},
 {'tm': ' 02', 'p': 1197.55},
 {'tm': ' 03', 'p': 1567.35},
 {'tm': ' 04', 'p': 1058.46},
 {'tm': ' 05', 'p': 867.94},
 {'tm': ' 06', 'p': 731.64},
 {'tm': ' 07', 'p': 767.06},
 {'tm': ' 08', 'p': 853.19},
 {'tm': ' 09', 'p': 896.51},
 {'tm': ' 10', 'p': 865.63},
 {'tm': ' 11', 'p': 883.03},
 {'tm': ' 12', 'p': 860.18},
 {'tm': ' 13', 'p': 861.46},
 {'tm': ' 14', 'p': 872.27},
 {'tm': ' 15', 'p': 851.84},
 {'tm': ' 16', 'p': 849.35},
 {'tm': ' 17', 'p': 810.46},
 {'tm': ' 18', 'p': 790.99},
 {'tm': ' 19', 'p': 769.93},
 {'tm': ' 20', 'p': 755.95},
 {'tm': ' 21', 'p': 697.25},
 {'tm': ' 22', 'p': 597.15},
 {'tm': ' 23', 'p': 549.28}]

## 7.每天每个城市起降数列表

### 时间—每个城市起降数列表<城市，起降数>

In [3]:
df.show(3)

+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+--------------------+
|arr_ap|arr_ct|   arr_tm| company|dep_ap|dep_ct|   dep_tm|   discount|flightNum| flight_dt|plane_sz| plane_type|price|    qry_dt|            rec_uuid|
+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+--------------------+
|      |  三亚| 00:00:00|吉祥航空|      |  上海| 00:00:00|3.1折经济舱|   HO1129|2019-09-12|      中|   空客A321|730.0|2019-09-11|2c0f8dab-56ff-4ea...|
|      |  三亚| 00:00:00|四川航空|      |  重庆| 00:00:00|3.1折经济舱|   3U8715|2019-09-12|      中|   空客A320|560.0|2019-09-11|7f3ff21f-5d3a-4a0...|
|      |  三亚| 00:00:00|山东航空|      |  济南| 00:00:00|2.3折经济舱|   SC4751|2019-09-13|      中|波音737-800|640.0|2019-09-12|cc08d14f-f89d-4eb...|
+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+----------

In [4]:
dd = df.select("arr_ct","dep_ct","flight_dt","price")
dd = dd.select("arr_ct","dep_ct","price",f.split(dd.flight_dt,"-").alias("sp_dt"))
dd = dd.withColumn("cnnt",f.abs(dd.price-dd.price+1))
dd.show(5)

+------+------+-----+--------------+----+
|arr_ct|dep_ct|price|         sp_dt|cnnt|
+------+------+-----+--------------+----+
|  三亚|  上海|730.0|[2019, 09, 12]| 1.0|
|  三亚|  重庆|560.0|[2019, 09, 12]| 1.0|
|  三亚|  济南|640.0|[2019, 09, 13]| 1.0|
|  三亚|  温州|530.0|[2019, 09, 21]| 1.0|
|  三亚|  重庆|530.0|[2019, 09, 12]| 1.0|
+------+------+-----+--------------+----+
only showing top 5 rows



In [5]:
dd_2 = dd.withColumn('year',dd["sp_dt"].getItem(0)).drop("line").drop("arr_ct").drop("price")
dd_2 = dd_2.withColumn('month',dd_2["sp_dt"].getItem(1))
dd_2 = dd_2.withColumn('day',dd_2["sp_dt"].getItem(2)).drop("sp_dt")
dd_2 = dd_2.filter(dd_2['year'] == '2020')
dd_2.show(5)

+------+----+----+-----+---+
|dep_ct|cnnt|year|month|day|
+------+----+----+-----+---+
|  上海| 1.0|2020|   05| 25|
|  贵阳| 1.0|2020|   06| 20|
|  福州| 1.0|2020|   01| 14|
|  南京| 1.0|2020|   01| 30|
|  南京| 1.0|2020|   01| 13|
+------+----+----+-----+---+
only showing top 5 rows



In [6]:
dd_3 = dd_2.groupBy("month","day","dep_ct").agg({"cnnt":"sum"})
dd_3.show(5)

+-----+---+------+---------+
|month|day|dep_ct|sum(cnnt)|
+-----+---+------+---------+
|   01| 21|  温州|    134.0|
|   05| 11|  上海|    533.0|
|   04| 15|  温州|     92.0|
|   04| 19|  成都|    338.0|
|   03| 03|  成都|    293.0|
+-----+---+------+---------+
only showing top 5 rows



In [7]:
dd_4 = dd_3.withColumn("date", f.concat_ws('-',dd_3['month'],dd_3['day']))
dd_4 = dd_4.drop("month").drop("day")
dd_4 = dd_4.withColumnRenamed("sum(cnnt)","linenum")
dd_4 = dd_4.withColumn("linenum", dd_4["linenum"].cast(IntegerType()))
dd_4.show(5)

+------+-------+-----+
|dep_ct|linenum| date|
+------+-------+-----+
|  温州|    134|01-21|
|  上海|    533|05-11|
|  温州|     92|04-15|
|  成都|    338|04-19|
|  成都|    293|03-03|
+------+-------+-----+
only showing top 5 rows



In [47]:
dd_5 = dd_4.groupBy("date").pivot("dep_ct").sum("linenum").na.fill(0)

### 全国每天航班数

In [12]:
dd_5_t = dd_4.groupBy("date").sum("linenum").na.fill(0)
dd_5_t = dd_5_t.withColumnRenamed("sum(linenum)","linenum")
dd_5_t.show(5)

+-----+-------+
| date|linenum|
+-----+-------+
|05-25|   6494|
|02-29|   1479|
|04-13|   5734|
|05-09|   5648|
|05-21|   6503|
+-----+-------+
only showing top 5 rows



In [11]:
prop = {
    'user':'root',
    'password':'1',
    'driver':'com.mysql.jdbc.Driver'
}
url = 'jdbc:mysql://localhost:3306/flight?useUnicode=true&characterEncoding=utf8'

In [13]:
dd_5_t.write.jdbc(url, 'total_flightnum_byday', properties=prop)

In [48]:
city = ['北京','长春','重庆','长沙','成都','石家庄',
        '广州','贵阳','桂林','海口','呼和浩特','三亚',
        '哈尔滨','济南','昆明','兰州','拉萨','武汉',
        '南宁','泉州','青岛','上海','沈阳','郑州',
        '深圳','天津','太原','乌鲁木齐','温州','珠海',
        '西安','厦门','西宁','银川','扬州','无锡',
        '大连','福州','合肥','杭州','南昌','南京']

In [49]:
dd_5_t = dd_5
for c in city:
    dd_5_t = dd_5_t.withColumn(c, dd_5[c].cast(IntegerType()))
dd_5_t.show(5)

+-----+----+----+--------+----+----+----+----+----+----+----+--------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+
| date|三亚|上海|乌鲁木齐|兰州|北京|南京|南宁|南昌|厦门|合肥|呼和浩特|哈尔滨|大连|天津|太原|广州|成都|扬州|拉萨|无锡|昆明|杭州|桂林|武汉|沈阳|泉州|济南|海口|深圳|温州|珠海|石家庄|福州|西宁|西安|贵阳|郑州|重庆|银川|长春|长沙|青岛|
+-----+----+----+--------+----+----+----+----+----+----+----+--------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+
|05-25| 144| 485|     131| 100| 370| 190| 116|  87| 180|  82|     103|    85| 131| 156|  88| 331| 361|  33|  48|  60| 248| 245|  42|  47| 117|  65| 130| 147| 351|  86|  94|    69| 105|  65| 262| 154| 184| 322|  86|  75| 164| 155|
|02-29|   0| 244|       0|   0| 214|   0|   0|   0| 106|   0|       0|     0|   0|   0|   0| 170|   0|   0|   0|   0|   0| 152|   0|   0|   0|  27|   0|   0| 208|  40|  4

In [51]:
dd_6 = dd_5_t.orderBy("date")
dd_7 = dd_6.toJSON()
ddc = dd_7.collect()
ddc[0]

'{"date":"01-01","三亚":208,"上海":630,"乌鲁木齐":163,"兰州":125,"北京":590,"南京":237,"南宁":139,"南昌":126,"厦门":233,"合肥":114,"呼和浩特":91,"哈尔滨":230,"大连":171,"天津":189,"太原":143,"广州":449,"成都":402,"扬州":55,"拉萨":54,"无锡":70,"昆明":357,"杭州":302,"桂林":64,"武汉":195,"沈阳":211,"泉州":94,"济南":151,"海口":231,"深圳":401,"温州":109,"珠海":119,"石家庄":93,"福州":119,"西宁":78,"西安":370,"贵阳":169,"郑州":228,"重庆":367,"银川":113,"长春":161,"长沙":195,"青岛":198}'

In [53]:
dDict = []
for i in range(len(ddc)):
    ret = {}
    linumlist = {}
    tmp = json.loads(ddc[i])
    for key in tmp :
        if key == 'date':
            ret['date'] = tmp[key]
        else :
            linumlist[key] = tmp[key]
    ret['linumlist']=str(linumlist)
    dDict.append(ret)
dDict[2]

{'date': '01-03',
 'linumlist': "{'三亚': 213, '上海': 598, '乌鲁木齐': 0, '兰州': 0, '北京': 617, '南京': 253, '南宁': 142, '南昌': 0, '厦门': 221, '合肥': 0, '呼和浩特': 0, '哈尔滨': 0, '大连': 0, '天津': 0, '太原': 0, '广州': 437, '成都': 397, '扬州': 52, '拉萨': 52, '无锡': 68, '昆明': 327, '杭州': 302, '桂林': 63, '武汉': 0, '沈阳': 0, '泉州': 98, '济南': 0, '海口': 225, '深圳': 418, '温州': 113, '珠海': 116, '石家庄': 0, '福州': 98, '西宁': 0, '西安': 0, '贵阳': 0, '郑州': 0, '重庆': 322, '银川': 0, '长春': 68, '长沙': 0, '青岛': 0}"}

In [54]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext,HiveContext
from pyspark.sql.types import *

schema = StructType([
    StructField("date", StringType(), True),
    StructField("linumlist", StringType(), True)])

ddf = sqlContext.createDataFrame(dDict,schema)
ddf.show(5)

+-----+------------------------+
| date|               linumlist|
+-----+------------------------+
|01-01|{'三亚': 208, '上海':...|
|01-02|{'三亚': 211, '上海':...|
|01-03|{'三亚': 213, '上海':...|
|01-04|{'三亚': 208, '上海':...|
|01-05|{'三亚': 198, '上海':...|
+-----+------------------------+
only showing top 5 rows



### 写数据库(city_flightnum_byday)

In [55]:
prop = {
    'user':'root',
    'password':'1',
    'driver':'com.mysql.jdbc.Driver'
}
url = 'jdbc:mysql://localhost:3306/flight?useUnicode=true&characterEncoding=utf8'

In [56]:
ddf.write.jdbc(url, 'city_flightnum_byday', properties=prop)

## 8.每个城市平均最低价格及所在最多的时间点

### 城市—最低价格—最低价格时间点

In [3]:
df.show(3)

+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+--------------------+
|arr_ap|arr_ct|   arr_tm| company|dep_ap|dep_ct|   dep_tm|   discount|flightNum| flight_dt|plane_sz| plane_type|price|    qry_dt|            rec_uuid|
+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+--------------------+
|      |  三亚| 00:00:00|吉祥航空|      |  上海| 00:00:00|3.1折经济舱|   HO1129|2019-09-12|      中|   空客A321|730.0|2019-09-11|2c0f8dab-56ff-4ea...|
|      |  三亚| 00:00:00|四川航空|      |  重庆| 00:00:00|3.1折经济舱|   3U8715|2019-09-12|      中|   空客A320|560.0|2019-09-11|7f3ff21f-5d3a-4a0...|
|      |  三亚| 00:00:00|山东航空|      |  济南| 00:00:00|2.3折经济舱|   SC4751|2019-09-13|      中|波音737-800|640.0|2019-09-12|cc08d14f-f89d-4eb...|
+------+------+---------+--------+------+------+---------+-----------+---------+----------+--------+-----------+-----+----------+----------

### 出发

In [65]:
dv = df.select("dep_tm","price","dep_ct",f.split(df.dep_tm,":").alias("sp_tm"))
dv = dv.withColumn('hour',dv["sp_tm"].getItem(0))
dv = dv.withColumn('minute',dv["sp_tm"].getItem(1)).drop("sp_tm")
dv = dv.withColumn("minutes", dv["minute"].cast(IntegerType()))
# dv = dv.withColumn("hours", dv["hour"].cast(IntegerType()))
dv.show(5)

+---------+-----+------+----+------+-------+
|   dep_tm|price|dep_ct|hour|minute|minutes|
+---------+-----+------+----+------+-------+
| 00:00:00|730.0|  上海|  00|    00|      0|
| 00:00:00|560.0|  重庆|  00|    00|      0|
| 00:00:00|640.0|  济南|  00|    00|      0|
| 00:00:00|530.0|  温州|  00|    00|      0|
| 00:00:00|530.0|  重庆|  00|    00|      0|
+---------+-----+------+----+------+-------+
only showing top 5 rows



In [66]:
dv_t = dv.select("*",f.split(dv.hour," ").alias("sp_tm"))
dv_t2 = dv_t.withColumn('ihour',dv_t["sp_tm"].getItem(1)).drop("sp_tm")
dv_t2.show(5)

+---------+-----+------+----+------+-------+-----+
|   dep_tm|price|dep_ct|hour|minute|minutes|ihour|
+---------+-----+------+----+------+-------+-----+
| 00:00:00|730.0|  上海|  00|    00|      0|   00|
| 00:00:00|560.0|  重庆|  00|    00|      0|   00|
| 00:00:00|640.0|  济南|  00|    00|      0|   00|
| 00:00:00|530.0|  温州|  00|    00|      0|   00|
| 00:00:00|530.0|  重庆|  00|    00|      0|   00|
+---------+-----+------+----+------+-------+-----+
only showing top 5 rows



In [68]:
dv_2 = dv_t2.withColumn("cnt",f.abs(dv_t2.price-dv_t2.price+1))
dv_2 = dv_2.groupBy("ihour","dep_ct").agg({"price":"sum","cnt":"sum","minutes":"sum"})
dv_2.show(5)

+-----+--------+----------+------------+--------+
|ihour|  dep_ct|sum(price)|sum(minutes)|sum(cnt)|
+-----+--------+----------+------------+--------+
|   08|乌鲁木齐| 2741750.0|       51185|  2223.0|
|   12|    大连| 1628540.0|       70780|  2199.0|
|   13|    南京| 1326985.0|       62650|  2039.0|
|   14|    海口| 2523711.0|       70665|  2701.0|
|   15|    沈阳| 2083463.0|       48520|  1926.0|
+-----+--------+----------+------------+--------+
only showing top 5 rows



In [69]:
dv_3 = dv_2.select("*",dv_2['sum(price)']/dv_2['sum(cnt)'],dv_2['sum(minutes)']/dv_2['sum(cnt)'])
dv_3 = dv_3.drop('sum(price)').drop('sum(cnt)').drop('sum(minutes)')
dv_3 = dv_3.select("*",f.bround("(sum(price) / sum(cnt))", scale=2).alias('avg_price')).drop("(sum(price) / sum(cnt))")
dv_3 = dv_3.select("*",f.bround("(sum(minutes) / sum(cnt))", scale=2).alias('avg_minute')).drop("(sum(minutes) / sum(cnt))")
dv_3.show(5)

+-----+--------+---------+----------+
|ihour|  dep_ct|avg_price|avg_minute|
+-----+--------+---------+----------+
|   08|乌鲁木齐|  1233.36|     23.03|
|   12|    大连|   740.58|     32.19|
|   13|    南京|    650.8|     30.73|
|   14|    海口|   934.36|     26.16|
|   15|    沈阳|  1081.76|     25.19|
+-----+--------+---------+----------+
only showing top 5 rows



In [70]:
dv_4 = dv_3.withColumn("minutes", dv_3["avg_minute"].cast(IntegerType()))
dv_4 = dv_4.withColumn("minute", dv_4["minutes"].cast("string"))
dv_4.show(5)

+-----+--------+---------+----------+-------+------+
|ihour|  dep_ct|avg_price|avg_minute|minutes|minute|
+-----+--------+---------+----------+-------+------+
|   08|乌鲁木齐|  1233.36|     23.03|     23|    23|
|   12|    大连|   740.58|     32.19|     32|    32|
|   13|    南京|    650.8|     30.73|     30|    30|
|   14|    海口|   934.36|     26.16|     26|    26|
|   15|    沈阳|  1081.76|     25.19|     25|    25|
+-----+--------+---------+----------+-------+------+
only showing top 5 rows



In [75]:
dv_5 = dv_4.select("ihour","dep_ct","avg_price","minute")
dvc = dv_5.collect()
dvc[0]

Row(ihour='08', dep_ct='乌鲁木齐', avg_price=1233.36, minute='23')

#### 开始选出每个城市的最小值

In [73]:
len(dvc)

828

In [61]:
no = ['0','1','2','3','4','5','6','7','8','9']

vDict = {}
for i in range(len(dvc)):
    dvc3 = dvc[i][3]
    if(dvc[i][3] in no):
        dvc3 = '0'+dvc3
    if dvc[i][1] not in vDict:
        vDict[dvc[i][1]] = (f'{dvc[i][0]}.{dvc3}',dvc[i][2])
    else :
        if vDict[dvc[i][1]][1] > dvc[i][2] :
            vDict[dvc[i][1]] = (f'{dvc[i][0]}.{dvc3}',dvc[i][2])
print(len(vDict))
vDict    

42


{'乌鲁木齐': ('22.11', 895.8),
 '大连': ('23.16', 432.71),
 '南京': ('23.11', 450.68),
 '海口': ('23.23', 578.21),
 '沈阳': ('22.19', 410.76),
 '福州': ('23.15', 506.17),
 '昆明': ('23.17', 510.61),
 '北京': ('23.14', 620.02),
 '济南': ('23.16', 538.52),
 '拉萨': ('23.45', 334.0),
 '温州': ('23.22', 436.39),
 '天津': ('23.18', 403.49),
 '贵阳': ('23.19', 489.67),
 '珠海': ('23.14', 570.52),
 '呼和浩特': ('22.33', 470.42),
 '长沙': ('22.22', 461.14),
 '西安': ('05.50', 350.0),
 '桂林': ('20.25', 436.45),
 '武汉': ('06.30', 375.0),
 '重庆': ('23.25', 457.62),
 '长春': ('06.19', 492.42),
 '石家庄': ('23.16', 459.51),
 '无锡': ('23.13', 544.02),
 '深圳': ('23.28', 627.0),
 '厦门': ('22.20', 439.15),
 '青岛': ('23.26', 468.02),
 '扬州': ('08.26', 346.96),
 '三亚': ('05.35', 615.0),
 '合肥': ('22.25', 480.12),
 '太原': ('01.05', 525.0),
 '南昌': ('00.25', 483.25),
 '杭州': ('23.18', 534.71),
 '郑州': ('01.50', 415.0),
 '广州': ('23.19', 668.46),
 '兰州': ('21.28', 592.08),
 '南宁': ('23.37', 429.24),
 '银川': ('23.37', 517.68),
 '上海': ('23.11', 581.39),
 '西宁': ('06.37'

In [62]:
vAns = []
for k in vDict:
    d = {}
    d['city'] = k
    d['clock'] = vDict[k][0]
    d['price'] = vDict[k][1]
    vAns.append(d)
vAns

[{'city': '乌鲁木齐', 'clock': '22.11', 'price': 895.8},
 {'city': '大连', 'clock': '23.16', 'price': 432.71},
 {'city': '南京', 'clock': '23.11', 'price': 450.68},
 {'city': '海口', 'clock': '23.23', 'price': 578.21},
 {'city': '沈阳', 'clock': '22.19', 'price': 410.76},
 {'city': '福州', 'clock': '23.15', 'price': 506.17},
 {'city': '昆明', 'clock': '23.17', 'price': 510.61},
 {'city': '北京', 'clock': '23.14', 'price': 620.02},
 {'city': '济南', 'clock': '23.16', 'price': 538.52},
 {'city': '拉萨', 'clock': '23.45', 'price': 334.0},
 {'city': '温州', 'clock': '23.22', 'price': 436.39},
 {'city': '天津', 'clock': '23.18', 'price': 403.49},
 {'city': '贵阳', 'clock': '23.19', 'price': 489.67},
 {'city': '珠海', 'clock': '23.14', 'price': 570.52},
 {'city': '呼和浩特', 'clock': '22.33', 'price': 470.42},
 {'city': '长沙', 'clock': '22.22', 'price': 461.14},
 {'city': '西安', 'clock': '05.50', 'price': 350.0},
 {'city': '桂林', 'clock': '20.25', 'price': 436.45},
 {'city': '武汉', 'clock': '06.30', 'price': 375.0},
 {'city': '重

### 到达

In [63]:
dc = df.select("dep_tm","price","arr_ct",f.split(df.arr_tm,":").alias("sp_tm"))
dc = dc.withColumn('hour',dc["sp_tm"].getItem(0))
dc = dc.withColumn('minute',dc["sp_tm"].getItem(1)).drop("sp_tm")
dc = dc.withColumn("minutes", dc["minute"].cast(IntegerType()))
# dv = dv.withColumn("hours", dv["hour"].cast(IntegerType()))
dc.show(5)

+---------+-----+------+----+------+-------+
|   dep_tm|price|arr_ct|hour|minute|minutes|
+---------+-----+------+----+------+-------+
| 00:00:00|730.0|  三亚|  00|    00|      0|
| 00:00:00|560.0|  三亚|  00|    00|      0|
| 00:00:00|640.0|  三亚|  00|    00|      0|
| 00:00:00|530.0|  三亚|  00|    00|      0|
| 00:00:00|530.0|  三亚|  00|    00|      0|
+---------+-----+------+----+------+-------+
only showing top 5 rows



In [64]:
dc_t = dc.select("*",f.split(dc.hour," ").alias("sp_tm"))
dc_t2 = dc_t.withColumn('ihour',dc_t["sp_tm"].getItem(1)).drop("sp_tm")
dc_t2.show(5)

+---------+-----+------+----+------+-------+-----+
|   dep_tm|price|arr_ct|hour|minute|minutes|ihour|
+---------+-----+------+----+------+-------+-----+
| 00:00:00|730.0|  三亚|  00|    00|      0|   00|
| 00:00:00|560.0|  三亚|  00|    00|      0|   00|
| 00:00:00|640.0|  三亚|  00|    00|      0|   00|
| 00:00:00|530.0|  三亚|  00|    00|      0|   00|
| 00:00:00|530.0|  三亚|  00|    00|      0|   00|
+---------+-----+------+----+------+-------+-----+
only showing top 5 rows



In [74]:
dc_2 = dc_t2.withColumn("cnt",f.abs(dc_t2.price-dc_t2.price+1))
dc_2 = dc_2.groupBy("ihour","arr_ct").agg({"price":"sum","cnt":"sum","minutes":"sum"})
dc_2.show(5)

+-----+--------+----------+------------+--------+
|ihour|  arr_ct|sum(price)|sum(minutes)|sum(cnt)|
+-----+--------+----------+------------+--------+
|   12|    大连| 1882452.0|       55080|  2422.0|
|   13|    南京| 2043076.0|       80655|  2584.0|
|   14|    海口| 1923775.0|       61245|  2044.0|
|   15|    沈阳| 2083995.0|       78430|  2283.0|
|   08|乌鲁木齐|   43010.0|         855|    24.0|
+-----+--------+----------+------------+--------+
only showing top 5 rows



In [77]:
dc_3 = dc_2.select("*",dc_2['sum(price)']/dc_2['sum(cnt)'],dc_2['sum(minutes)']/dc_2['sum(cnt)'])
dc_3 = dc_3.drop('sum(price)').drop('sum(cnt)').drop('sum(minutes)')
dc_3 = dc_3.select("*",f.bround("(sum(price) / sum(cnt))", scale=2).alias('avg_price')).drop("(sum(price) / sum(cnt))")
dc_3 = dc_3.select("*",f.bround("(sum(minutes) / sum(cnt))", scale=2).alias('avg_minute')).drop("(sum(minutes) / sum(cnt))")
dc_3.show(5)

+-----+--------+---------+----------+
|ihour|  arr_ct|avg_price|avg_minute|
+-----+--------+---------+----------+
|   12|    大连|   777.23|     22.74|
|   13|    南京|   790.66|     31.21|
|   14|    海口|   941.18|     29.96|
|   15|    沈阳|   912.83|     34.35|
|   08|乌鲁木齐|  1792.08|     35.62|
+-----+--------+---------+----------+
only showing top 5 rows



In [78]:
dc_4 = dc_3.withColumn("minutes", dc_3["avg_minute"].cast(IntegerType()))
dc_4 = dc_4.withColumn("minute", dc_4["minutes"].cast("string"))
dc_4.show(5)

+-----+--------+---------+----------+-------+------+
|ihour|  arr_ct|avg_price|avg_minute|minutes|minute|
+-----+--------+---------+----------+-------+------+
|   12|    大连|   777.23|     22.74|     22|    22|
|   13|    南京|   790.66|     31.21|     31|    31|
|   14|    海口|   941.18|     29.96|     29|    29|
|   15|    沈阳|   912.83|     34.35|     34|    34|
|   08|乌鲁木齐|  1792.08|     35.62|     35|    35|
+-----+--------+---------+----------+-------+------+
only showing top 5 rows



In [79]:
dc_5 = dc_4.select("ihour","arr_ct","avg_price","minute")
dcc = dc_5.collect()
dcc[0]

Row(ihour='12', arr_ct='大连', avg_price=777.23, minute='22')

In [80]:
len(dcc)

819

In [82]:
noc = ['0','1','2','3','4','5','6','7','8','9']

cDict = {}
for i in range(len(dcc)):
    dcc_3 = dcc[i][3]
    if(dcc[i][3] in noc):
        dcc_3 = '0'+dcc_3
    if dcc[i][1] not in cDict:
        cDict[dcc[i][1]] = (f'{dcc[i][0]}.{dcc_3}',dcc[i][2])
    else :
        if cDict[dcc[i][1]][1] > dcc[i][2] :
            cDict[dcc[i][1]] = (f'{dcc[i][0]}.{dcc_3}',dcc[i][2])
print(len(cDict))
cDict    

42


{'大连': ('07.55', 516.89),
 '南京': ('08.40', 512.39),
 '海口': ('08.29', 622.22),
 '沈阳': ('07.55', 577.1),
 '乌鲁木齐': ('01.24', 1179.55),
 '北京': ('08.46', 660.4),
 '福州': ('07.48', 277.25),
 '昆明': ('07.34', 366.38),
 '济南': ('08.34', 517.4),
 '拉萨': ('08.38', 1079.21),
 '温州': ('09.37', 560.9),
 '珠海': ('07.33', 412.0),
 '贵阳': ('08.42', 584.48),
 '天津': ('08.35', 486.55),
 '长沙': ('08.41', 409.54),
 '呼和浩特': ('20.22', 665.96),
 '西安': ('07.46', 388.35),
 '青岛': ('08.32', 468.23),
 '武汉': ('07.54', 405.24),
 '重庆': ('07.52', 400.4),
 '桂林': ('04.10', 415.0),
 '长春': ('07.51', 506.67),
 '深圳': ('08.35', 580.06),
 '厦门': ('08.42', 529.5),
 '无锡': ('08.41', 593.97),
 '石家庄': ('08.45', 486.23),
 '扬州': ('08.51', 270.0),
 '三亚': ('09.23', 703.46),
 '合肥': ('08.32', 529.3),
 '太原': ('08.38', 537.63),
 '南昌': ('09.27', 518.28),
 '杭州': ('08.37', 547.86),
 '兰州': ('07.35', 351.6),
 '广州': ('08.34', 587.94),
 '郑州': ('07.55', 250.0),
 '南宁': ('07.50', 257.36),
 '上海': ('08.42', 642.98),
 '银川': ('07.45', 358.93),
 '西宁': ('07.38', 

In [83]:
cAns = []
for k in cDict:
    d = {}
    d['city'] = k
    d['clock'] = cDict[k][0]
    d['price'] = cDict[k][1]
    cAns.append(d)
cAns

[{'city': '大连', 'clock': '07.55', 'price': 516.89},
 {'city': '南京', 'clock': '08.40', 'price': 512.39},
 {'city': '海口', 'clock': '08.29', 'price': 622.22},
 {'city': '沈阳', 'clock': '07.55', 'price': 577.1},
 {'city': '乌鲁木齐', 'clock': '01.24', 'price': 1179.55},
 {'city': '北京', 'clock': '08.46', 'price': 660.4},
 {'city': '福州', 'clock': '07.48', 'price': 277.25},
 {'city': '昆明', 'clock': '07.34', 'price': 366.38},
 {'city': '济南', 'clock': '08.34', 'price': 517.4},
 {'city': '拉萨', 'clock': '08.38', 'price': 1079.21},
 {'city': '温州', 'clock': '09.37', 'price': 560.9},
 {'city': '珠海', 'clock': '07.33', 'price': 412.0},
 {'city': '贵阳', 'clock': '08.42', 'price': 584.48},
 {'city': '天津', 'clock': '08.35', 'price': 486.55},
 {'city': '长沙', 'clock': '08.41', 'price': 409.54},
 {'city': '呼和浩特', 'clock': '20.22', 'price': 665.96},
 {'city': '西安', 'clock': '07.46', 'price': 388.35},
 {'city': '青岛', 'clock': '08.32', 'price': 468.23},
 {'city': '武汉', 'clock': '07.54', 'price': 405.24},
 {'city': '